In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
#from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

import nltk
from nltk.corpus import stopwords
#from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from nltk.probability import FreqDist
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer

In [54]:
# Load the data
eo_df = pd.read_csv('../data/eo_cleaned.csv', index_col=0)
laws_df = pd.read_csv('../data/laws_cleaned.csv', index_col=0)

# add target
eo_df['target'] = 0
laws_df['target'] = 1

# add year column
eo_df['year'] = pd.to_datetime(eo_df['date']).dt.year
#eo_df['year'] = eo_df['year'].astype(int)
laws_df['year'] = pd.to_datetime(laws_df['Date of Introduction']).dt.year


In [55]:
eo_df[eo_df['year'].isna()]

,order_no,title,text,date,cleaned_text,text_string,target,year
337,10338,NaN,by virtue of the authority vested in me by sec...,NaN,"['virtue', 'authority', 'vested', 'section', '...",virtue authority vested section 507 mutual sec...,0,NaN
338,10339,NaN,by virtue of the authority vested in me by sec...,NaN,"['virtue', 'authority', 'vested', 'section', '...",virtue authority vested section 611 act entitl...,0,NaN
548,11515,NaN,"march 13, 1970 by virtue of the authority ves...",NaN,"['march', '13', '1970', 'virtue', 'authority',...",march 13 1970 virtue authority vested presiden...,0,NaN
2052,13704,NaN,federal register part iii the president exe...,NaN,"['federal', 'register', 'part', 'iii', 'presid...",federal register part iii president executive ...,0,NaN
2134,13866,NaN,please see this document's talk page for detai...,NaN,"['please', 'see', 'document', 'talk', 'page', ...",please see document talk page detail verificat...,0,NaN
2158,13962,NaN,"executive order 13962 of december 8, 2020 ensu...",NaN,"['executive', 'order', '13962', 'december', '8...",executive order 13962 december 8 2020 ensuring...,0,NaN


In [56]:
# order 103338 https://www.presidency.ucsb.edu/documents/executive-order-10338-coordination-procedures-under-section-507-the-mutual-security-act
eo_df.at[337, 'date'] = 'April 4, 1952'
eo_df.at[337, 'year'] = 1952
eo_df.at[337, 'title'] = 'Coordination Procedures Under Section 507 of the Mutual Security Act of 1951'

# order 10339 https://www.archives.gov/federal-register/executive-orders/1952.html#10339
eo_df.at[338, 'date'] = 'April 4, 1952'
eo_df.at[338, 'year'] = 1952
eo_df.at[338, 'title'] = 'Extensions of Time Relating to the Disposition of Certain Housing'

# order 11515
eo_df.at[548, 'date'] = 'March 13, 1970'
eo_df.at[548, 'year'] = 1970

# order 13704 https://www.govinfo.gov/content/pkg/DCPD-201500563/pdf/DCPD-201500563.pdf
eo_df.at[2052, 'date'] = 'August 17, 2015'
eo_df.at[2052, 'year'] = 2015
eo_df.at[2052, 'title'] = 'Presidential Innovation Fellows Program'


# order 13866 https://www.presidency.ucsb.edu/documents/executive-order-13866-adjustments-certain-rates-pay
eo_df.at[2134, 'date'] = 'March 28, 2019'
eo_df.at[2134, 'year'] = 2019
eo_df.at[2134, 'title'] = 'Adjustments of Certain Rates of Pay'

# order 13867 https://www.federalregister.gov/documents/2019/04/15/2019-07645/issuance-of-permits-with-respect-to-facilities-and-land-transportation-crossings-at-the
eo_df.at[2135, 'date'] = 'April 10, 2019'
eo_df.at[2135, 'year'] = 2019
eo_df.at[2135, 'title'] = 'Issuance of Permits with Respect to Facilities and Land Transportation Crossings at the International Boundaries of the United States'

# order 13962 https://www.presidency.ucsb.edu/documents/executive-order-13962-ensuring-access-united-states-government-covid-19-vaccines
eo_df.at[2158, 'date'] = 'December 8, 2020'
eo_df.at[2158, 'year'] = 2020
eo_df.at[2158, 'title'] = 'Ensuring Access to United States Government COVID-19 Vaccines'

In [57]:
eo_df[eo_df['year'].isna()]

,order_no,title,text,date,cleaned_text,text_string,target,year


In [58]:
eo_df.isna().sum()

order_no        0
title           9
text            0
date            0
cleaned_text    0
text_string     7
target          0
year            0
dtype: int64

In [59]:
laws_df.isna().sum()

Legislation Number      0
URL                     0
Congress                0
Title                   0
Sponsor                 0
Date of Introduction    0
Chamber                 0
Bill Number             0
Text                    0
cleaned_text            0
text_string             0
target                  0
year                    0
dtype: int64

# EDA

In [71]:
fdist = FreqDist()

eo_df['tokens'] = [text.split(" ") for text in eo_df['text_string']]
for list in eo_df['tokens']:
    for word in list:
        fdist[word] += 1

fdist.most_common(50)

AttributeError: 'float' object has no attribute 'split'